# **Scraping SatuSehat (PeduliLindungi) News**

Salah satu cara untuk melakukan web scraping adalah dengan menggunakan `Selenium`. Beberapa kelebihan dari Selenium, yaitu:
- Selenium dapat berinteraksi dengan elemen di halaman web, seperti mengklik tombol dan menavigasi antar halaman yang sangat berguna untuk mengakses konten yang dimuat secara dinamis.
- Selenium dapat digunakan pada berbagai browser, salah satunya adalah Google Chrome.


## Set Up

Sebelum memulai lebih lanjut, kita harus menyiapkan lingkungan di Ubuntu dengan mengunduh dan menginstal Google Chrome dan ChromeDriver agar Selenium dapat mengotomatisasi browser.

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 2s (139 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping a

In [ ]:
# Install Libraries
!pip install chromedriver-autoinstaller
!pip install dateparser

In [ ]:
# Import library
import re
import csv
import sys
import time
import dateparser
import pandas as pd
import chromedriver_autoinstaller
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Konfigurasi opsi Chrome
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # Agar berjalan di mode tanpa antarmuka
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/128/chromedriver'

# **Scraping Link Artikel pada Berita di Google**

In [ ]:
# Inisialisasi list untuk menyimpan semua data
all_articles = set()

In [ ]:
# Fungsi untuk memodifikasi link Google News
def rearrange_news_link(url):
  # Memisahkan URL berdasarkan pola 'start=' diikuti angka dan '&sa='
  url_split = re.split(r"start=(\d+)&sa=", url)
  # Mengembalikan URL yang telah diubah
  return url_split[0] + 'start=', '&sa=' + url_split[-1]

In [ ]:
# Fungsi untuk hanya mengambil bagian utama dari link berita yang didapatkan
def get_google_main_url(url):
  return re.findall(r"^(.*?)(?=&)", url)[0]

In [ ]:
def scrape_google_news_link(result_set, number, url):
    # Memisahkan URL menjadi bagian yang bisa diproses
    url1, url2 = rearrange_news_link(url)

    # Mendapatkan URL utama dari URL yang telah dipisahkan
    main_url = get_google_main_url(url1)
    print(f"Mengakses URL: {main_url}")

    # Menginisialisasi driver Chrome dengan opsi yang sudah ditentukan
    driver = webdriver.Chrome(options=chrome_options)
    article_xpath = "//div[contains(@class, 'MjjYud')]/div"

    try:
        # Mengambil artikel dari URL yang diberikan, sebanyak <number> artikel
        for i in range(0, number, 10):
            # Membuat URL untuk halaman artikel berikutnya
            page = url1 + str(i) + url2
            driver.get(page)
            time.sleep(2)  # Memberi waktu untuk halaman memuat

            # Menemukan elemen yang berisi artikel
            articles = driver.find_element(By.XPATH, article_xpath)
            articles = articles.find_elements(By.CLASS_NAME, "SoaBEf")
            print(f"Mengumpulkan artikel dari {i+1} hingga {i+10}")

            # Mengambil link, judul, dan tanggal dari setiap artikel
            for article in articles:
                # Mengambil link dari elemen <a>
                a = article.find_element(By.TAG_NAME, "a").get_attribute("href")

                # Mengambil tanggal dari elemen dengan kelas 'OSrXXb'
                date = dateparser.parse(
                    date_string=article.find_element(By.CLASS_NAME, "OSrXXb").text,
                    languages=['en', 'id'])

                # Mengambil judul dari elemen dengan kelas 'n0jPhd ynAwRc MBeuO nDgy9d'
                #title = article.find_element(By.CLASS_NAME, "n0jPhd").text

                # Menambahkan tuple (link, tanggal) ke hasil
                result_set.add(tuple([a, date.strftime('%Y-%m-%d')]))

    except NoSuchElementException as e:
        # Jika elemen artikel tidak ditemukan
        print('Halaman artikel sudah tidak tersedia')

    finally:
        # Menutup driver setelah proses selesai
        driver.close()
        print(f"Proses selesai. Total link artikel unik yang berhasil diambil: {len(result_set)}")
        print('----------------------------------------------------------')


In [ ]:
# Link Google News yang akan di-scraping
# satu sehat
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=satu+sehat&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWIIiJ31AXdt_1uPlcuRXCrWbOCJRPg:1725864313300&ei=eZneZv-IEoKgseMPiN7GyQs&start=0&sa=N&ved=2ahUKEwi_qczKobWIAxUCUGwGHQivMbk4ChDy0wN6BAgDEAQ&biw=1536&bih=703&dpr=1.25')
# peduli lindungi
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=peduli+lindungi&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWII8LRnsJNcGclJCB8Yvnvf7td5KGQ:1725864448748&ei=AJreZra2Ld-R4-EP4YLriAM&start=0&sa=N&ved=2ahUKEwi2tpeLorWIAxXfyDgGHWHBGjE4ChDy0wN6BAgDEAQ&biw=1536&bih=703&dpr=1.25')
# peduli lindungi covid 19
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=peduli+lindungi+covid+19&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWIJyBmmyIspHbsCsZmAnDpYCI7bcJg:1725864485869&ei=JZreZp3gNLLB4-EP65bUkQc&start=0&sa=N&ved=2ahUKEwjdhvGcorWIAxWy4DgGHWsLNXI4ChDy0wN6BAgEEAQ&biw=1536&bih=703&dpr=1.25')
# layanan peduli lindungi
scrape_google_news_link(all_articles, 1000, 'https://www.google.com/search?q=layanan+peduli+lindungi&sca_esv=449428b1edf18ef7&tbm=nws&sxsrf=ADLYWIK-xWiGNOvjvOTzKmIHXYcHTmeSfA:1725864527340&ei=T5reZoW_FMiN4-EPzbq24Q4&start=0&sa=N&ved=2ahUKEwjFotSworWIAxXIxjgGHU2dLew4ChDy0wN6BAgBEAQ&biw=1536&bih=703&dpr=1.25')

Mengakses URL: https://www.google.com/search?q=satu+sehat
Mengumpulkan artikel dari 1 hingga 10
Mengumpulkan artikel dari 11 hingga 20
Mengumpulkan artikel dari 21 hingga 30
Mengumpulkan artikel dari 31 hingga 40
Mengumpulkan artikel dari 41 hingga 50
Mengumpulkan artikel dari 51 hingga 60
Mengumpulkan artikel dari 61 hingga 70
Mengumpulkan artikel dari 71 hingga 80
Mengumpulkan artikel dari 81 hingga 90
Mengumpulkan artikel dari 91 hingga 100
Mengumpulkan artikel dari 101 hingga 110
Mengumpulkan artikel dari 111 hingga 120
Mengumpulkan artikel dari 121 hingga 130
Mengumpulkan artikel dari 131 hingga 140
Mengumpulkan artikel dari 141 hingga 150
Mengumpulkan artikel dari 151 hingga 160
Mengumpulkan artikel dari 161 hingga 170
Mengumpulkan artikel dari 171 hingga 180
Mengumpulkan artikel dari 181 hingga 190
Mengumpulkan artikel dari 191 hingga 200
Mengumpulkan artikel dari 201 hingga 210
Mengumpulkan artikel dari 211 hingga 220
Mengumpulkan artikel dari 221 hingga 230
Mengumpulkan artike

In [ ]:
# Contoh isi link_set
for i, data in enumerate(all_articles):
  if (i == 5):
    break
  print(data)

('http://www.thejakartapost.com/indonesia/2022/04/19/govt-brushes-off-privacy-concerns-over-covid-19-app.html', '2022-04-19')
('https://kumparan.com/berita-update/cara-ganti-nomor-hp-di-pedulilindungi-lewat-aplikasi-whatsapp-1x3dVLUXZBO', '2021-12-06')
('https://katadata.co.id/berita/nasional/63c91014c0b2a/tak-semua-rumah-sakit-siap-terhubung-digitalisasi-satu-sehat', '2023-01-19')
('https://mediaindonesia.com/humaniora/561697/platform-satu-sehat-diharap-jadi-standar-kesehatan-nasional', '2023-02-28')
('https://www.tribunnews.com/bisnis/2024/09/01/cegah-penyebaran-mpox-di-ri-citilink-minta-penumpang-rute-internasional-isi-aplikasi-satu-sehat', '2024-09-02')


# **Ubah Set Menjadi DataFrame**

In [ ]:
newsLink_df = pd.DataFrame(data=all_articles, columns=['link', 'tanggal'])

In [ ]:
# Konversi tanggal menjadi datetime, urutkan dari terbaru ke terlama
newsLink_df['tanggal'] = pd.to_datetime(newsLink_df['tanggal'], format='%Y-%m-%d')
newsLink_df = newsLink_df.sort_values(by='tanggal', ascending=False, ignore_index=True)

In [ ]:
newsLink_df

,link,tanggal
0,https://www.gresiksatu.com/ribuan-warga-antusi...,2024-09-09
1,https://rri.co.id/kesehatan/959404/aplikasi-sa...,2024-09-07
2,https://www.kompas.id/baca/humaniora/2024/09/0...,2024-09-07
3,https://beritajateng.tv/antisipasi-penyakir-mp...,2024-09-07
4,https://radarbangkalan.jawapos.com/news/231506...,2024-09-06
...,...,...
512,https://sehatnegeriku.kemkes.go.id/baca/umum/2...,2019-09-28
513,https://sehatnegeriku.kemkes.go.id/baca/umum/2...,2019-01-29
514,https://health.detik.com/kebugaran/d-4277353/c...,2018-10-29
515,https://sehatnegeriku.kemkes.go.id/baca/rilis-...,2016-05-05


# **Simpan Link Artikel ke CSV**

In [ ]:
output_file = 'link_berita_satusehat.csv'
newsLink_df.to_csv(output_file, index=False)

message = f"Successfully output the results to the {output_file} file"
print('-' * len(message))
print(message)
print(f"Successfully got {len(all_articles)} unique article links")

---------------------------------------------------------------------
Successfully output the results to the link_berita_satusehat.csv file
Successfully got 517 unique article links
